# Relacijska drevesa v Pythonu: RE3PY

Družini modelov, ki sta zelo trpežni in prilagodljivi:

- mreže,
- drevesa.

Danes bomo spoznali relacijska drevesa.

## Primerjava navadnih in relacijskih dreves za klasifikacijo


|  | navadno drevo | relacijsko drevo |
| --- | --- | --- |
| vhod | vektor vrednosti `x` | referenca na vrstico ciljne tabele (`id`) |
| notranja vozlišča | testi oblike `x[i]` $\in S$ | ?? |
| listi | napovedi (večinski razred učnih primerov) | isto |

Pri numeričnih atributih je množica $S$ interval $(-\infty, \theta)$, sicer pa je podmnožica (končne) zaloge vrednosti (npr. `{A, AB, 0}`).

## Predstavitev podatkov

Tabele si mislimo kot dejstva:

```
business_category(biz1, zdravilišče)
business_category(biz2, gostilna)
...
business_stars(biz1, 4)
business_stars(biz2, 4)
...
hours_open(biz1, ponedeljek, 7)
hours_open(biz1, torek, 7)
...
review_user_id(rev1, user1)
review_biz_id(rev1, biz1)
...
```

## Kako prilagoditi teste?

Tako, kot smo to naredili "na roke" (približno). Obstajata dve pristopa:

- tak, ki temelji na logiki in uporablja agregate,
- tak, ki temelji na logiki in preverja samo obstoj,
- tak, ki temelji na sprehodih po grafu in uporablja agregate.

### Logika in agregati

Želimo test oblike `f(id)` $\in S$, kje je `f(id)` vrednost neke značilke `f` za primer `id`
(npr. povprečna zabavnost uporabnika, ki je napisal recenzijo za dani poslovni obrat).

Ideja:

1. Najdi vse recenzije za poslovni obrat `id`: iščemo dejstva oblike `review_biz_id(X1, id)`
2. Za vsako tak `X1` najdi vse avtorje (natanko enega): `review_user_id(X1, X2)`
3. Za vsak tak `X2` najdi zabavnost: `users_funny(X2, X3)`
4. Izračunaj povprečje vrednosti `X3`

oz. natančneje

1. Najdi vse $r_1(X_{1,1}, \dots, X_{1, n_1}) \land r_2(X_{2,1}, \dots, X_{1, n_2})\land \dots \land r_1(X_{k,1}, \dots, X_{k, n_1})$
2. Izberi $X_{i,j}$ in agregiraj njegove vrednosti.

Primerni agregati: števec, min, max, sum, ..., modus

Pozor: $r_1(X, Y, Z)$ je nekaj drugega kot $r_1(X, Y, X)$ ... Ogromno možnosti.

### Logika in preverjanje obstoja

Kot zgoraj, a uporabi le števec in $\theta = 0{,}5$

### Sprehodi po grafu

Sledimo neki predpisani meta-poti $r_1 - r_2 - \dots - r_k$ (se znebimo spremenljivk)

# Ogled kode

In [2]:
from re3py.learners.tree import DecisionTree
from re3py.learners.random_forest import RandomForest
# from re3py.learners.boosting import GradientBoosting
from re3py.learners.core.heuristic import HeuristicGini
from re3py.data.data_and_statistics import Dataset
from re3py.data.task_settings import Settings
from re3py.ranking.ensemble_ranking import EnsembleRanking
from re3py.utilities.cross_validation import create_folds
from sklearn.metrics import accuracy_score

## Priprava podatkov

In [6]:
data_name = "yelp_small"


print(data_name)

experiment_dir = f"data/datasets/{data_name}/"
descriptive = experiment_dir + f'{data_name}_descriptive.txt'
target = experiment_dir + f'{data_name}_target.txt'
s_file = experiment_dir + f'{data_name}.s'
s = Settings(s_file)
d = Dataset(s_file, descriptive, target)
tree_params = {'heuristic': HeuristicGini(),
               'max_number_internal_nodes': 100,
               'max_number_atom_tests': 1, #max dolzina sprehoda
               'allowed_atom_tests': s.get_atom_tests_structured(),
               'allowed_aggregators': s.get_aggregates(),
               'minimal_examples_in_leaf': 1,
               'max_number_of_evaluated_tests_per_node': 1000,
               'max_depth': 20,
               'only_existential': False} 


d_train, d_test = next(create_folds(d, n_folds=4))

yelp_small


## Učenje modelov

In [7]:
t = DecisionTree(**tree_params)
t.fit(d_train)
t.dump_to_text('rel_drevo.txt')
# t.dump_to_bin('rel_drevo.bin')

rf = RandomForest(4, **tree_params)
# GradientBoosting(2, **tree_params)  #  DecisionTree(**tree_params)  #
rf.fit(d_train)
rf.dump_to_text('rel_rf.txt')
# rf.dump_to_bin('rel_rf.bin')

rf.compute_ranking(EnsembleRanking.genie3).dump_to_text("genie3_rf.txt")

  Building node on depth 1
    Building node on depth 2
      Building node on depth 3
        Building node on depth 4
          Building node on depth 5
            Building node on depth 6
              Building node on depth 7
                Building node on depth 8
                  Building node on depth 9
                    Building node on depth 10
                      Building node on depth 11
                        Building node on depth 12
                        Building node on depth 12
                          Building node on depth 13
                          Building node on depth 13
                      Building node on depth 11
                    Building node on depth 10
                      Building node on depth 11
                      Building node on depth 11
                  Building node on depth 9
                    Building node on depth 10
                    Building node on depth 10
                      Building node on depth 11
              

## Napovedovanje in preizkušanje

In [8]:
true_values = [e.target_part for e in d_test.target_data]
predicted_values = [[model.predict(e) for e in d_test.target_data] for model in [t, rf]]

for predictions in predicted_values:
    print(accuracy_score(true_values, predictions))


0.8354430379746836
0.8164556962025317
